In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler, Sampler

In [7]:
train_X = []
train_y = []

with open('perm_train.txt') as file:
    for line in file:
        data = line.split()
        train_y.append(int(data.pop()))
        train_X.append(list(map(int, data)))

test_X = []
test_y = []

with open('perm_test.txt') as file:
    for line in file:
        data = line.split()
        test_y.append(int(data.pop()))
        test_X.append(list(map(int, data)))

train_X = torch.tensor(train_X, dtype=torch.float)
train_y = torch.tensor(train_y, dtype=torch.long)
test_X = torch.tensor(test_X, dtype=torch.float)
test_y = torch.tensor(test_y, dtype=torch.long)

In [8]:
train_X -= torch.mean(train_X, dim=0)
test_X -= torch.mean(test_X, dim=0)

In [9]:
data_train = []
for i in range(1000):
    data_train.append((train_X[i], train_y[i]))

In [10]:
data_train[0]

(tensor([ 2.3020e+00, -8.1700e-01, -2.0560e+00,  3.7700e-01, -5.7890e+00,
         -7.8720e+00, -2.9590e+00, -2.8220e+00,  3.1330e+00,  4.1730e+00,
          2.3570e+00,  2.2500e-01, -2.9260e+00,  4.0520e+00,  1.0340e+00,
         -6.8290e+00,  1.2020e+00, -9.0900e-01,  4.9450e+00,  7.1690e+00,
          3.1620e+00,  3.1520e+00,  4.2330e+00, -6.1510e+00, -2.7850e+00,
          3.9760e+00,  4.2700e+00,  3.2910e+00,  3.6500e-01, -7.3600e-01,
          6.9540e+00, -1.8510e+00,  4.0760e+00,  2.1860e+00,  1.2740e+00,
         -4.7600e+00,  7.2070e+00, -8.2000e-01, -5.9620e+00,  4.2470e+00,
          4.9430e+00,  5.3300e+00,  1.2720e+00,  2.1490e+00,  1.4090e+00,
          2.0480e+00,  6.1430e+00, -8.7200e-01, -1.7430e+00, -8.0150e+00,
         -3.0450e+00, -1.9520e+00,  4.1740e+00,  2.0710e+00,  1.3130e+00,
          1.1120e+00,  6.1950e+00, -3.8750e+00, -2.8280e+00, -3.0970e+00,
         -8.3500e-01,  3.0960e+00,  3.9890e+00, -2.8220e+00, -1.9510e+00,
          3.0910e+00, -5.0580e+00,  2.

In [114]:
batch_size = 1000

data_size = 1000
validation_split = .2
split = int(validation_split * data_size)
indices = np.arange(data_size)
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                         sampler=val_sampler)

In [117]:
sls = 100
nn_model = nn.Sequential(
            nn.Linear(1000, sls),
            nn.ReLU(inplace=True),
            nn.Linear(sls, sls),
            nn.ReLU(inplace=True),
            nn.Linear(sls, 2), 
         )
nn_model.type(torch.FloatTensor)

# We will minimize cross-entropy between the ground truth and
# network predictions using an SGD optimizer
loss = nn.CrossEntropyLoss().type(torch.FloatTensor)
optimizer = optim.SGD(nn_model.parameters(), lr=2, weight_decay=1e-3)

In [118]:
def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):    
    loss_history = []
    train_history = []
    val_history = []
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
    for epoch in range(num_epochs):
        scheduler.step()
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            prediction = model(x)    
            loss_value = loss(prediction, y)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        ave_loss = loss_accum / (i_step + 1)
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
        
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
    """
    Computes accuracy on the dataset wrapped in a loader
    
    Returns: accuracy as a float value between 0 and 1
    """
    model.eval() # Evaluation mode
    # TODO: Implement the inference of the model on all of the batches from loader,
    #       and compute the overall accuracy.
    # Hint: torch doesn't have a dedicated argmax function,
    #       but you can use torch.max instead (see the documentation).
    
    correct_samples = 0
    total_samples = 0
    
    for x, y in loader:
        predictions = model(x)
        
        _, indices = torch.max(predictions, 1)
        
        correct_samples += torch.sum(indices == y)
        total_samples += y.shape[0]
    
    accuracy = float(correct_samples) / total_samples
    return accuracy

loss_history, train_history, val_history = train_model(nn_model, train_loader, val_loader, loss, optimizer, 30)

Average loss: 0.721501, Train accuracy: 0.481250, Val accuracy: 0.525000
Average loss: 2.357935, Train accuracy: 0.505000, Val accuracy: 0.475000
Average loss: 14.796508, Train accuracy: 0.495000, Val accuracy: 0.475000
Average loss: 0.800126, Train accuracy: 0.495000, Val accuracy: 0.475000
Average loss: 0.702538, Train accuracy: 0.495000, Val accuracy: 0.475000
Average loss: 0.695843, Train accuracy: 0.495000, Val accuracy: 0.475000
Average loss: 0.689957, Train accuracy: 0.497500, Val accuracy: 0.475000
Average loss: 0.683440, Train accuracy: 0.526250, Val accuracy: 0.550000
Average loss: 0.654913, Train accuracy: 0.740000, Val accuracy: 0.605000
Average loss: 0.599556, Train accuracy: 0.863750, Val accuracy: 0.665000
Average loss: 0.507157, Train accuracy: 0.901250, Val accuracy: 0.690000
Average loss: 0.467553, Train accuracy: 0.912500, Val accuracy: 0.690000
Average loss: 0.438441, Train accuracy: 0.915000, Val accuracy: 0.690000
Average loss: 0.427165, Train accuracy: 0.921250, 